# Final Assignment

## The goal

The problem that I will be solving during this capstone project is regarding a trip to the United States. I hypothetically have saved a lot of money, such that I can go there, but my goal is very specific. My first name is “Anne”, so the goal of my trip is to visit all cities with the name Anne in their name. To make it even more specific, I really want to let my family know that I have visited those places, so that is why I am looking for the post office in each of these cities. This is the point where data science will come in, since my goal is to make a list of all cities that include the name Anne and to find the post officies within these cities. Because based on this, I can plan my trip to the United States.
	One last note, I will make this algorithm in a very general way, such that people with similar interests can use it to plan their trips too. For example, when people want to visit all restaurants in cities that include an ‘X’. That is why I will first define the string that need to be in the city's name and the venue that I would like to find:


In [184]:
StringToBeFound = "Anne"
String_Venue = "Post Office"

## The Code

### Importing Packages

First, all necessary packages will be imported

In [185]:
import pandas as pd
import requests
import folium
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize

### Importing Data

Now, the data will be imported. The main data source that will be used is from the website OpenDataSoft (https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/table/) and this data source includes 43.191 cities in the United States with their respective zip codes and longitude and latitude values. I will use this dataset by finding all rows with “Anne” and their respective longitude and latitude values. These will be used in FourSquare to find the postal offices nearby these cities.

In [186]:
path = "us-zip-code-latitude-and-longitude.xlsx"
DF = pd.DataFrame(columns=["Zipcode","City","State","Latitude","Longitude","Timezone","Daylight Savings","Geopoint"])
DF = pd.read_excel(path)
DF

,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,55795,Willow River,MN,46317812.0,-9284315.0,-6,1,"46.317812,-92.84315"
1,45388,Yorkshire,OH,40328535.0,-8447938.0,-5,1,"40.328535,-84.47938"
2,39483,Foxworth,MS,31218509.0,-8990761.0,-6,1,"31.218509,-89.90761"
3,31503,Waycross,GA,31205194.0,-8237534.0,-5,1,"31.205194,-82.37534"
4,45833,Delphos,OH,40841409.0,-8434178.0,-5,1,"40.841409,-84.34178"
...,...,...,...,...,...,...,...,...
43186,25612,Chauncey,WV,3776662.0,-8198755.0,-5,1,"37.76662,-81.98755"
43187,59444,Galata,MT,48635319.0,-11124083.0,-7,1,"48.635319,-111.24083"
43188,60532,Lisle,IL,41789079.0,-8808536.0,-6,1,"41.789079,-88.08536"
43189,61638,Peoria,IL,40796887.0,-89611146.0,-6,1,"40.796887,-89.611146"


Once the data is imported, the columns that are not of use for this particular case will be deleted from the dataframe.

In [187]:
DF.drop(columns=["Zip","Daylight savings time flag","Timezone"], axis=1)

,City,State,Latitude,Longitude,geopoint
0,Willow River,MN,46317812.0,-9284315.0,"46.317812,-92.84315"
1,Yorkshire,OH,40328535.0,-8447938.0,"40.328535,-84.47938"
2,Foxworth,MS,31218509.0,-8990761.0,"31.218509,-89.90761"
3,Waycross,GA,31205194.0,-8237534.0,"31.205194,-82.37534"
4,Delphos,OH,40841409.0,-8434178.0,"40.841409,-84.34178"
...,...,...,...,...,...
43186,Chauncey,WV,3776662.0,-8198755.0,"37.76662,-81.98755"
43187,Galata,MT,48635319.0,-11124083.0,"48.635319,-111.24083"
43188,Lisle,IL,41789079.0,-8808536.0,"41.789079,-88.08536"
43189,Peoria,IL,40796887.0,-89611146.0,"40.796887,-89.611146"


Now, we have our data on all the cities in the United States. This dataframe will be used throughout this project.

### Finding all cities with "Anne"

The next step is to find all the cities with "Anne" in their name. To achieve this, first, a new dataframe will be created (DF_Anne)

In [188]:
# Make a new dataframe to which we can add the cities with "Anne"
DF_Anne = pd.DataFrame(columns=["City","State","Latitude","Longitude"])

Now, this dataframe will be filled with all the cities that include "Anne". This is done through looping over each row and appending rows to the new dataframe DF_Anne in cases where "Anne" is found in the city's name. To make it more general, you can also change the string to be found to another name and try it for that one (as is showed at the top).

In [189]:
TotalRows = DF.shape[0]

for i in range(0,TotalRows):
    CityName = DF["City"][i]
    if (CityName.find(StringToBeFound) != -1):
        #print(CityName)
        df = DF.iloc[i,1:5]
        Series_Object = pd.Series(DF.iloc[i,1:5],index=DF.columns)
        DF_Anne = DF_Anne.append(Series_Object,ignore_index=True)
        #DF_Anne.append(row)
DF_Anne.drop(columns=["Daylight savings time flag","Timezone","Zip","geopoint"], axis=1)

,City,State,Latitude,Longitude
0,Princess Anne,MD,38205272.0,-7570169.0
1,Saint Anne,IL,41045785.0,-8768963.0
2,Queen Anne,MD,38945143.0,-7596915.0
3,Annemanie,AL,32048079.0,-8726226.0


This table above includes all cities with the name Anne, based on the use of the longitude and latitude we can get the venues at these cities.

### Use FourSquare to find Venues

First, I will define my credentials

In [190]:
CLIENT_ID = 'LSKLK1UK3CZZXETMMH10LH4PFCMMK2CDNLXLT5WLI22YKMQ2' # your Foursquare ID
CLIENT_SECRET = 'FB4BBCYIN4YQCU0NCM2DH1UZBZ201W00CRSIQUMAA1HBEISA' # your Foursquare Secret
#code you get J0M10DG5GJV2CLLK24XRKKDCETBTHFL2BJX5MH5HJW4PJZRY
ACCESS_TOKEN = 'ICIKRIPRTRSWCKSME0J52KMJAAWS12MHUS2SSA3DPUFDUBW1' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

Now, we will define a couple of functions such that we can use them over and over again. The first function will be to get the coordinates of a certain location

In [191]:
address = 'Princess Anne, MD'

def GetCoordinates(Address):
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))
    return(latitude,longitude)

GetCoordinates(address)

The geograpical coordinate of Princess Anne, MD are 38.2028428, -75.6920008.


(38.2028428, -75.6920008)

The next function is to find all categories of the venues within a certain radius of this city. For now this radius is set at 10 km, but can be changed easily. Next to that, since we want to find all venues of these cities, we have chosen a limit of 150. All cities with the name "Anne" have less than 150 venues, so we can find them easily.

In [192]:
radius = 10000
LIMIT = 150

def FindAllVenues(latitude,longitude,radius,limit):
    #define an empty dataframe
    Venues_DF = pd.DataFrame(columns=["id","Category","Location"])
    
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            radius, 
            LIMIT)

    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    results

    #make a dataframe from the results (JSON)
    DF_Venues = pd.DataFrame.from_dict(results)

    #extract the venue types from the venues
    for i in range(0,DF_Venues.shape[0]):
        #print(DF_Venues.iloc[i,1]['location']['formattedAddress'])
        Series_Object1 = pd.Series((DF_Venues.iloc[i,1]['categories'][0]['id'],DF_Venues.iloc[i,1]['categories'][0]['name'],DF_Venues.iloc[i,1]['location']['formattedAddress']),index=["id","Category","Location"])
        Venues_DF = Venues_DF.append(Series_Object1,ignore_index=True)

    return(Venues_DF)


## Get the end result - Cities with Venues

Now, we will use the functions that we have determined above to loop over all venues and all cities (that we have found) to find all the venues that we are looking for. 

In [193]:
#Create a new dataframe
DF_SportVenues = pd.DataFrame(columns=["id","category","address"])

#loop over the enitre dataframe to find everything
for j in range(0,DF_Anne.shape[0]):
    #define the address of the city, such that we can get the venues out of FourSqyare
    address = str(DF_Anne["City"][j] + "," + DF_Anne["State"][j])
    latitude,longitude = GetCoordinates(address)
    DF_Venues = FindAllVenues(latitude,longitude,radius, limit)
    #loop over all venues that we have found to find the ones with the correct category
    for i in range(0,DF_Venues.shape[0]):
        if DF_Venues["Category"][i].find(String_Venue) != -1:
            SeriesObject3 = pd.Series((DF_Venues["id"][i],DF_Venues["Category"][i],DF_Venues["Location"][i]),index=["id","category","address"])
            DF_SportVenues = DF_SportVenues.append(SeriesObject3,ignore_index=True)

print(DF_SportVenues)

The geograpical coordinate of Princess Anne,MD are 38.2028428, -75.6920008.
The geograpical coordinate of Saint Anne,IL are 41.021996, -87.72044267569515.
The geograpical coordinate of Queen Anne,MD are 38.919559, -75.9528827.
The geograpical coordinate of Annemanie,AL are 32.0518121, -87.0391522.
                         id     category  \
0  4bf58dd8d48988d172941735  Post Office   
1  4bf58dd8d48988d172941735  Post Office   
2  4bf58dd8d48988d172941735  Post Office   
3  4bf58dd8d48988d172941735  Post Office   
4  4bf58dd8d48988d172941735  Post Office   

                                             address  
0  [31676 Eden Allen Rd (Rt 13), Eden, MD 21822, ...  
1  [702 Saint Charles St, Beaverville, IL 60912, ...  
2  [22043 Main St, Hillsboro, MD 21641, United St...  
3  [2560 County Road 4, Minter, AL 36761, United ...  
4  [8545 County Road 59, Furman, AL 36741, United...  


## End result

The end result is an dataframe with the category and the location that we need to visit. In case we would like to know more about the venues, we can use the ID.

In [194]:
DF_SportVenues

,id,category,address
0,4bf58dd8d48988d172941735,Post Office,"[31676 Eden Allen Rd (Rt 13), Eden, MD 21822, ..."
1,4bf58dd8d48988d172941735,Post Office,"[702 Saint Charles St, Beaverville, IL 60912, ..."
2,4bf58dd8d48988d172941735,Post Office,"[22043 Main St, Hillsboro, MD 21641, United St..."
3,4bf58dd8d48988d172941735,Post Office,"[2560 County Road 4, Minter, AL 36761, United ..."
4,4bf58dd8d48988d172941735,Post Office,"[8545 County Road 59, Furman, AL 36741, United..."
